In [1]:
!gcloud auth login --update-adc

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=xVSRbIuA8DAL7xGySEWbd3G7cNlOEv&access_type=offline&code_challenge=uxjXyGsWFO5HMPqQp6Zj04VnzvouWmSnVzD6jtBXzDE&code_challenge_method=S256


Application Default Credentials (ADC) were updated.

You are now logged in as [biyan.bahtiar.1@gmail.com].
Your current project is [b-508911].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
!gcloud config get-value project

b-508911


In [3]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from multiprocessing import Pool

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from joblib import dump,load
from google.cloud import storage, bigquery
import json
from datetime import datetime

import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

from time import time
import numpy as np

In [4]:
bq_client = bigquery.Client()
storage_client = storage.Client()

In [5]:
datasets = [obj.dataset_id for obj in list(bq_client.list_datasets())]
tables = {dataset: [table.table_id for table in bq_client.list_tables(dataset)] for dataset in datasets}

print(tables)

{'ml_ops': ['bank_campaign_model_metrics'], 'test_dataset_iykra': ['cost_test', 'test_table', 'view1'], 'test_schema': ['us_states']}


In [ ]:
project_id = 'b-508911'
dataset_id = 'test_dataset_iykra'
table_id = 'test_table'

query_all_test_table = f"""
select *
from {project_id}.{dataset_id}.{table_id}
where Date >= date_sub(current_date(), interval 1 year)
"""

test_table_df = bq_client.query(query_all_test_table).to_dataframe()
test_table_df.head()

In [ ]:
test_table_df.info()

# Data Preprocessing

In [ ]:
min_date = test_table['Date'].min()
max_date = test_table['Date'].max()

print(f'min_date:{min_date}, max_date:{max_date}')

In [ ]:
cols, names = list(), list()

df = test_table_df[['Date', 'Warehouse_ID', 'Product_ID', 'Discount_Impact', 'Sales_Event', 'Daily_Sales']]
df = df.sort_values(by=['Warehouse_ID', 'Product_ID', 'Date'])

tes = pd.DataFrame()

for i in range(30, 0, -1):
    cols.append(df.shift(i))
    names += [('%s(t-%d)' % (col, i)) for col in df.columns]
    tes = pd.concat(cols, axis=1)
    tes.columns = names

display(tes)

In [ ]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# Data Preprocessing
def preprocess_data(data):

    # Filter columns
    data = data[['Date', 'Warehouse_ID', 'Product_ID', 'Promotion', 'Discount', 'Sales']]

    # Sort data
    data = data.sort_values(by=['Warehouse_ID', 'Product_ID', 'Date'])

    # Filter None results and combine
    results = [r for r in results if r is not None]

    if not results:
        raise ValueError("No valid sequences generated!")

    sequences = np.concatenate([r[0] for r in results])
    targets = np.concatenate([r[1] for r in results])

    print(f"Generated {len(sequences)} sequences")
    return sequences, targets

In [ ]:


# Custom dataset class with memory optimization
class SalesDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.FloatTensor(features)
        self.targets = torch.FloatTensor(targets)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# LSTM Model with performance optimizations
class SalesLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(SalesLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )

        # Added batch normalization for better training stability
        self.batch_norm = nn.BatchNorm1d(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.batch_norm(out[:, -1, :])
        out = self.fc(out)
        return out

# Optimized preprocessing for a single store-item group
def process_group(group_data):
    try:
        store, item = group_data[0]
        group = group_data[1]

        if len(group) < 120:  # Minimum required length
            return None

        # Normalize numerical features
        scaler = StandardScaler()
        group['sales'] = scaler.fit_transform(group[['sales']])
        group['discount'] = scaler.fit_transform(group[['discount']])

        sequences = []
        targets = []

        # Create sequences with larger step size
        step_size = 3  # Create sequence every 3 days
        for i in range(0, len(group) - 119, step_size):  # 119 = window_size + horizon - 1
            sequence = group.iloc[i:i+30]  # window_size = 30
            target = group.iloc[i+30:i+120]['sales'].values  # 90 days prediction

            feature_vector = np.column_stack((
                sequence['sales'].values,
                sequence['promotion'].values,
                sequence['discount'].values,
                sequence['store'].values,
                sequence['item'].values
            ))

            sequences.append(feature_vector)
            targets.append(target)

        return np.array(sequences), np.array(targets)
    except Exception as e:
        print(f"Error processing store {store}, item {item}: {str(e)}")
        return None

# Optimized training function
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs, device):
    train_losses = []
    valid_losses = []

    scaler = torch.cuda.amp.GradScaler()  # For mixed precision training

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        # Use tqdm for progress bar
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for batch_features, batch_targets in train_pbar:
            batch_features = batch_features.to(device)
            batch_targets = batch_targets.to(device)

            # Mixed precision training
            with torch.cuda.amp.autocast():
                outputs = model(batch_features)
                loss = criterion(outputs, batch_targets)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_train_loss += loss.item()
            train_pbar.set_postfix({'loss': loss.item()})

        model.eval()
        total_valid_loss = 0

        valid_pbar = tqdm(valid_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Valid]')
        with torch.no_grad():
            for batch_features, batch_targets in valid_pbar:
                batch_features = batch_features.to(device)
                batch_targets = batch_targets.to(device)

                outputs = model(batch_features)
                loss = criterion(outputs, batch_targets)
                total_valid_loss += loss.item()
                valid_pbar.set_postfix({'loss': loss.item()})

        avg_train_loss = total_train_loss / len(train_loader)
        avg_valid_loss = total_valid_loss / len(valid_loader)

        train_losses.append(avg_train_loss)
        valid_losses.append(avg_valid_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}]: '
              f'Train Loss: {avg_train_loss:.4f}, '
              f'Valid Loss: {avg_valid_loss:.4f}')

    return train_losses, valid_losses

def main():
    # Hyperparameters
    HIDDEN_SIZE = 128
    NUM_LAYERS = 2
    BATCH_SIZE = 256  # Increased batch size
    NUM_EPOCHS = 50
    LEARNING_RATE = 0.001
    N_SAMPLES = 1000  # Number of store-item combinations to use

    # Preprocess data
    sequences, targets = preprocess_data(df, n_samples=N_SAMPLES)

    # Split data
    X_train, X_valid, y_train, y_valid = train_test_split(
        sequences, targets, test_size=0.2, random_state=42
    )

    # Create datasets and dataloaders
    train_dataset = SalesDataset(X_train, y_train)
    valid_dataset = SalesDataset(X_valid, y_valid)

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=4,  # Parallel data loading
        pin_memory=True  # Faster data transfer to GPU
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=BATCH_SIZE,
        num_workers=4,
        pin_memory=True
    )

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    input_size = sequences.shape[2]
    output_size = targets.shape[1]

    model = SalesLSTM(
        input_size=input_size,
        hidden_size=HIDDEN_SIZE,
        num_layers=NUM_LAYERS,
        output_size=output_size
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    # Add learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3, verbose=True
    )

    # Train model
    train_losses, valid_losses = train_model(
        model, train_loader, valid_loader,
        criterion, optimizer, NUM_EPOCHS, device
    )

    return model, train_losses, valid_losses

if __name__ == "__main__":
    model, train_losses, valid_losses = main()